### <절대 모멘텀>
1. 파생변수 'STD-YM' 생성 -> 인덱스에서 년-월을 추출해 대입
2. 'STD-YM'별 마지막 날의 데이터들을 month_last_df에 추가(단순 행결합)
3. 전월의 수정종가 값을 가진 파생변수 생성
4. 전년도의 수정종가 값을 가진 파생변수 생성
5. 전월의 데이터와 전년도의 데이터를 이용해 거래 내역 생성
6. 수익률 계산

In [139]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings

In [140]:
# warning 메시지 출력 필터링
warnings.filterwarnings('ignore')

In [141]:
df = pd.read_csv('../../csv/AMZN.csv')

In [142]:
if 'Date' in df.columns:
    df.set_index('Date', inplace=True)

In [143]:
# index를 시계열로 변환
df.index = pd.to_datetime(df.index)

In [144]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [145]:
# df에 STD-YM 컬럼 생성하여 인덱스에서 년도-월 데이터 추출하여 대입
df['STD-YM'] = df.index.strftime('%Y-%m')

In [146]:
df.head(2)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1997-05-15,2.43750,2.500000,1.927083,1.958333,1.958333,72156000,1997-05
1997-05-16,1.96875,1.979167,1.708333,1.729167,1.729167,14700000,1997-05


In [147]:
# 빈 데이터프레임에 월말의 데이터들을 행결합
# STD-YM에서 다음 행과 현재 행이 다르다면
flag = df['STD-YM'] != df['STD-YM'].shift(-1)
df.loc[flag, ]

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1997-05-30,1.500000,1.510417,1.479167,1.500000,1.500000,2594400,1997-05
1997-06-30,1.510417,1.598958,1.479167,1.541667,1.541667,2746800,1997-06
1997-07-31,2.437500,2.437500,2.333333,2.395833,2.395833,1454400,1997-07
1997-08-29,2.364583,2.375000,2.322917,2.338542,2.338542,722400,1997-08
1997-09-30,4.000000,4.348958,3.802083,4.338542,4.338542,5254800,1997-09
...,...,...,...,...,...,...,...
2019-02-28,1635.250000,1651.770020,1633.829956,1639.829956,1639.829956,3025900,2019-02
2019-03-29,1786.579956,1792.859985,1776.630005,1780.750000,1780.750000,3320800,2019-03
2019-04-30,1930.099976,1935.709961,1906.949951,1926.520020,1926.520020,3506000,2019-04


In [148]:
# 기준년월의 유니크 값을 따로 생성
month_list = df['STD-YM'].unique()
month_list

array(['1997-05', '1997-06', '1997-07', '1997-08', '1997-09', '1997-10',
       '1997-11', '1997-12', '1998-01', '1998-02', '1998-03', '1998-04',
       '1998-05', '1998-06', '1998-07', '1998-08', '1998-09', '1998-10',
       '1998-11', '1998-12', '1999-01', '1999-02', '1999-03', '1999-04',
       '1999-05', '1999-06', '1999-07', '1999-08', '1999-09', '1999-10',
       '1999-11', '1999-12', '2000-01', '2000-02', '2000-03', '2000-04',
       '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10',
       '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04',
       '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10',
       '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04',
       '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10',
       '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04',
       '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10',
       '2003-11', '2003-12', '2004-01', '2004-02', 

In [149]:
month_last_df = pd.DataFrame()

for month in month_list:
	# flag = df['STD-YM'] == month
	# data = df.loc[flag, ].tail()
	data = df.loc[month, ].tail(1)
	month_last_df = pd.concat(
		[month_last_df, data], axis=0
	)

In [150]:
month_last_df = month_last_df[['Adj Close', 'STD-YM']]
month_last_df.head()

,Adj Close,STD-YM
Date,,
1997-05-30,1.500000,1997-05
1997-06-30,1.541667,1997-06
1997-07-31,2.395833,1997-07
1997-08-29,2.338542,1997-08
1997-09-30,4.338542,1997-09


In [151]:
# 전월의 수정종가 파생변수(BF-1M) 생성
month_last_df['BF-1M'] = month_last_df['Adj Close'].shift(1).fillna(0)
# 매도의 수정종가 파생변수(BF-12M) 생성
month_last_df['BF-12M'] = month_last_df['Adj Close'].shift(12).fillna(0)

In [152]:
month_last_df.head(3)

,Adj Close,STD-YM,BF-1M,BF-12M
Date,,,,
1997-05-30,1.500000,1997-05,0.000000,0.0
1997-06-30,1.541667,1997-06,1.500000,0.0
1997-07-31,2.395833,1997-07,1.541667,0.0


- 거래 내역 추가
	- (전월의 수정종가) / (전년도의 수정종가) - 1 값이 0보다 크고 무한대가 아닌 경우가 매수 타이밍
	- 모멘텀 인덱스가 위의 조건에 부합할 경우 df의 trade 컬럼의 해당 월에 구매 내역 추가

In [153]:
df['trade'] = ''

# month_last_df의 인덱스를 기준으로 반복문 실행
for idx in month_last_df.index:
	signal= ''

	# 절대 모멘텀 인덱스 계산
	momentum_index = \
		(month_last_df.loc[idx, 'BF-1M'] / month_last_df.loc[idx, 'BF-12M']) - 1
	# print(momentum_index)
	# break
	# 조건식: 모멘텀 인덱스가 0보다 크고 무한대가 아닌
	# if문 한줄에 작성
	# 참인 경우 데이터 if 조건식 else 거짓인 경우 데이터
	flag = True if (momentum_index > 0) & (momentum_index != np.inf) else False
	
	# 조건(flag)이 참이라면
	if flag:
		signal = 'buy'
	print(f'날짜: {idx}, 모멘텀인덱스: {momentum_index}, signal: {signal}')
	df.loc[idx: , 'trade'] = signal

날짜: 1997-05-30 00:00:00, 모멘텀인덱스: nan, signal: 
날짜: 1997-06-30 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-07-31 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-08-29 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-09-30 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-10-31 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-11-28 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1997-12-31 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1998-01-30 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1998-02-27 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1998-03-31 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1998-04-30 00:00:00, 모멘텀인덱스: inf, signal: 
날짜: 1998-05-29 00:00:00, 모멘텀인덱스: 4.0972219999999995, signal: buy
날짜: 1998-06-30 00:00:00, 모멘텀인덱스: 3.7635124835648686, signal: buy
날짜: 1998-07-31 00:00:00, 모멘텀인덱스: 5.9391314002269775, signal: buy
날짜: 1998-08-31 00:00:00, 모멘텀인덱스: 6.902003042921615, signal: buy
날짜: 1998-09-30 00:00:00, 모멘텀인덱스: 2.217286590748689, signal: buy
날짜: 1998-10-30 00:00:00, 모멘텀인덱스: 2.659836174415487, signal: buy
날짜: 1998-11-30 00:00:00, 모멘텀인덱스: 4.10858569696969

In [154]:
df

,Open,High,Low,Close,Adj Close,Volume,STD-YM,trade
Date,,,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000,1997-05,
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000,1997-05,
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800,1997-05,
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200,1997-05,
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200,1997-05,
...,...,...,...,...,...,...,...,...
2019-06-18,1901.349976,1921.670044,1899.790039,1901.369995,1901.369995,3895700,2019-06,buy
2019-06-19,1907.839966,1919.579956,1892.469971,1908.790039,1908.790039,2895300,2019-06,buy
2019-06-20,1933.329956,1935.199951,1905.800049,1918.189941,1918.189941,3217200,2019-06,buy


In [155]:
df['trade'].value_counts()

trade
buy    4033
       1530
Name: count, dtype: int64

In [156]:
# 수익률 계산
# 매수일자: 전날의 trade가 ''이고, 오늘의 trade가 'buy'인 상태
# 매도일자: 전날의 trade가 'buy'이고, 오늘의 trade가 ''인 상태

df['rtn'] = 1

for idx in df.index:
    # 매수의 조건식
    if (df.shift().loc[idx, 'trade'] == '') & (df.loc[idx, 'trade'] == 'buy'):
        buy = df.loc[idx, 'Adj Close']		# 구매가는 해당 날짜의 수정종가(Adj Close)
        print(f'매수일: {idx}, 매수가: {buy}')
    # 매도의 조건식
    elif (df.shift().loc[idx, 'trade'] == 'buy') & (df.loc[idx, 'trade'] == ''):
        sell = df.loc[idx, 'Adj Close']		# 매도가는 해당 날짜의 수정종가(Adj Close)
        rtn = sell / buy
        df.loc[idx, 'rtn'] = rtn
        print(f'매도일: {idx}, 매도가: {sell}, 수익률: {rtn}')

매수일: 1998-05-29 00:00:00, 매수가: 7.34375
매도일: 2000-03-31 00:00:00, 매도가: 67.0, 수익률: 9.12340425531915
매수일: 2002-02-28 00:00:00, 매수가: 14.1
매도일: 2002-04-30 00:00:00, 매도가: 16.690001, 수익률: 1.183688014184397
매수일: 2002-06-28 00:00:00, 매수가: 16.25
매도일: 2004-08-31 00:00:00, 매도가: 38.139999, 수익률: 2.347076861538462
매수일: 2005-02-28 00:00:00, 매수가: 35.18
매도일: 2005-03-31 00:00:00, 매도가: 34.27, 수익률: 0.9741330301307563
매수일: 2005-08-31 00:00:00, 매수가: 42.700001
매도일: 2006-05-31 00:00:00, 매도가: 34.610001, 수익률: 0.8105386461232167
매수일: 2006-06-30 00:00:00, 매수가: 38.68
매도일: 2006-07-31 00:00:00, 매도가: 26.889999, 수익률: 0.6951912874870734
매수일: 2007-02-28 00:00:00, 매수가: 39.139999
매도일: 2008-07-31 00:00:00, 매도가: 76.339996, 수익률: 1.9504342859078763
매수일: 2009-06-30 00:00:00, 매수가: 83.660004
매도일: 2012-03-30 00:00:00, 매도가: 202.509995, 수익률: 2.420630950483818
매수일: 2012-04-30 00:00:00, 매수가: 231.899994
매도일: 2014-10-31 00:00:00, 매도가: 305.459991, 수익률: 1.31720568737919
매수일: 2015-03-31 00:00:00, 매수가: 372.100006


In [157]:
# 누적곱 연산 과정
acc_rtn = 1
for i in df.index:
    rtn = df.loc[i, 'rtn']
    acc_rtn *= rtn
acc_rtn
    

np.float64(86.52294619753461)

In [158]:
# 누적곱(누적수익률)
df['acc_rtn'] = df['rtn'].cumprod()

In [159]:
df.iloc[-1, ]['Adj Close'] / df.iloc[0, ]['Adj Close']		# buyandhold 수익률

np.float64(974.2744757914)

In [160]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume,STD-YM,trade,rtn,acc_rtn
Date,,,,,,,,,,
1997-05-15,2.4375,2.5,1.927083,1.958333,1.958333,72156000,1997-05,,1.0,1.0


In [161]:
df2 = df.drop(
    ['trade', 'rtn', 'acc_rtn'],
    axis=1
)

In [162]:
df2.head(1)

,Open,High,Low,Close,Adj Close,Volume,STD-YM
Date,,,,,,,
1997-05-15,2.4375,2.5,1.927083,1.958333,1.958333,72156000,1997-05


In [163]:
df2['trade'] = ''

# month_last_df의 인덱스를 기준으로 반복문 실행
for idx in month_last_df.index:
	signal= ''

	# 절대 모멘텀 인덱스 계산
	momentum_index = \
		(month_last_df.loc[idx, 'BF-1M'] / month_last_df.loc[idx, 'BF-12M']) - 1
	
	flag = True if (momentum_index > 0) & (momentum_index != np.inf) else False
	
	# 조건(flag)이 참이라면
	if flag:
		# 모멘텀 인덱스가 구매 신호 주면 해당 월말부터 마지막 데이터까지 buy로 채워줌
		# df2.loc[idx: , 'trade'] = 'buy'
		# 모멘텀 인덱스가 구매 신호 주면 해당 월말 다음 행의 STD-YM을 추출해
		# 해당 조건에 맞는 인덱스를 필터링해 trade에 buy 대입
		std_ym = df2.shift(-1).loc[idx, 'STD-YM']
		df2.loc[std_ym, 'trade'] = 'buy'
		print(f'날짜: {idx}, 모멘텀인덱스: {momentum_index}')

날짜: 1998-05-29 00:00:00, 모멘텀인덱스: 4.0972219999999995
날짜: 1998-06-30 00:00:00, 모멘텀인덱스: 3.7635124835648686
날짜: 1998-07-31 00:00:00, 모멘텀인덱스: 5.9391314002269775
날짜: 1998-08-31 00:00:00, 모멘텀인덱스: 6.902003042921615
날짜: 1998-09-30 00:00:00, 모멘텀인덱스: 2.217286590748689
날짜: 1998-10-30 00:00:00, 모멘텀인덱스: 2.659836174415487
날짜: 1998-11-30 00:00:00, 모멘텀인덱스: 4.108585696969697
날짜: 1998-12-31 00:00:00, 모멘텀인덱스: 5.373444406535729
날짜: 1999-01-29 00:00:00, 모멘텀인덱스: 9.889830041367455
날짜: 1999-02-26 00:00:00, 모멘텀인덱스: 8.112012513661687
날짜: 1999-03-31 00:00:00, 모멘텀인덱스: 7.9879482574779725
날짜: 1999-04-30 00:00:00, 모멘텀인덱스: 10.260218474559935
날짜: 1999-05-28 00:00:00, 모멘텀인덱스: 10.714893617021277
날짜: 1999-06-30 00:00:00, 모멘텀인덱스: 2.5714285714285716
날짜: 1999-07-30 00:00:00, 모멘텀인덱스: 2.385569456976576
날짜: 1999-08-31 00:00:00, 모멘텀인덱스: 2.5843284438048584
날짜: 1999-09-30 00:00:00, 모멘텀인덱스: 2.3426652933541874
날짜: 1999-10-29 00:00:00, 모멘텀인덱스: 2.7933762940069613
날짜: 1999-11-30 00:00:00, 모멘텀인덱스: 1.20703125
날짜: 1999-12-31 00:00:00, 모멘텀

In [164]:
df2['trade'].value_counts()

trade
buy    4033
       1531
Name: count, dtype: int64

In [165]:
# 수익률 계산
# 매수일자: 전날의 trade가 ''이고, 오늘의 trade가 'buy'인 상태
# 매도일자: 전날의 trade가 'buy'이고, 오늘의 trade가 ''인 상태

df2['rtn'] = 1

for idx in df2.index:
    # 매수의 조건식
    if (df2.shift().loc[idx, 'trade'] == '') & (df2.loc[idx, 'trade'] == 'buy'):
        buy = df2.loc[idx, 'Adj Close']		# 구매가는 해당 날짜의 수정종가(Adj Close)
        print(f'매수일: {idx}, 매수가: {buy}')
    # 매도의 조건식
    elif (df2.shift().loc[idx, 'trade'] == 'buy') & (df2.loc[idx, 'trade'] == ''):
        sell = df2.loc[idx, 'Adj Close']		# 매도가는 해당 날짜의 수정종가(Adj Close)
        rtn = sell / buy
        df2.loc[idx, 'rtn'] = rtn
        print(f'매도일: {idx}, 매도가: {sell}, 수익률: {rtn}')

매수일: 1998-06-01 00:00:00, 매수가: 7.135417
매도일: 2000-04-03 00:00:00, 매도가: 63.5625, 수익률: 8.908028780938801
매수일: 2002-03-01 00:00:00, 매수가: 15.39
매도일: 2002-05-01 00:00:00, 매도가: 16.49, 수익률: 1.0714749837556854
매수일: 2002-07-01 00:00:00, 매수가: 13.55
매도일: 2004-09-01 00:00:00, 매도가: 38.240002, 수익률: 2.8221403690036895
매수일: 2005-03-01 00:00:00, 매수가: 35.389999
매도일: 2005-04-01 00:00:00, 매도가: 34.009998, 수익률: 0.961005904521218
매수일: 2005-09-01 00:00:00, 매수가: 41.84
매도일: 2006-06-01 00:00:00, 매도가: 35.07, 수익률: 0.8381931166347992
매수일: 2006-07-03 00:00:00, 매수가: 38.610001
매도일: 2006-08-01 00:00:00, 매도가: 26.32, 수익률: 0.6816886640329277
매수일: 2007-03-01 00:00:00, 매수가: 38.849998
매도일: 2008-08-01 00:00:00, 매도가: 75.75, 수익률: 1.949807050183117
매수일: 2009-07-01 00:00:00, 매수가: 81.599998
매도일: 2012-04-02 00:00:00, 매도가: 198.050003, 수익률: 2.427083429585378
매수일: 2012-05-01 00:00:00, 매수가: 230.039993
매도일: 2014-11-03 00:00:00, 매도가: 305.720001, 수익률: 1.32898630804601
매수일: 2015-04-01 00:00:00, 매수가: 370.26001


In [166]:
df2['rtn'].cumprod()

Date
1997-05-15     1.000000
1997-05-16     1.000000
1997-05-19     1.000000
1997-05-20     1.000000
1997-05-21     1.000000
                ...    
2019-06-19    93.024067
2019-06-20    93.024067
2019-06-21    93.024067
2019-06-24    93.024067
NaT           93.024067
Name: rtn, Length: 5564, dtype: float64

### <절대 모멘텀 함수화>
1. STD-YM 컬럼을 생성하는 함수 (create_YM)
	- 매개변수 2개
		- _df: 데이터프레임(필수)
		- _col: 기준이 되는 컬럼(기본값 'Adj Close')
	1. 데이터프레임 복사본 생성
	2. 컬럼에 Date가 존재하면 index로 변경
	3. index를 시계열로 변경
	4. 결측치, 무한대 값 제거
	5. 기준 컬럼을 제외하고 나머지 제거
	6. 'STD-YM' 컬럼을 생성해 인덱스에서 년도-월 데이터를 추출해 대입
	7. 수정된 데이터프레임 반환

In [184]:
def create_YM(_df, _col = 'Adj Close'):
	# 데이터프레임 복사본 생성
	df = _df.copy()

	# 'Date'가 컬럼에 존재한다면 index로 변경
	if 'Date' in df.columns:
		df.set_index('Date', inplace=True)

	# 인덱스를 시계열로 변경
	df.index = pd.to_datetime(df.index)
    
	# 결측치, 무한대 값 제거
	flag = df.isin( [np.nan, np.inf, -np.inf] ).any(axis=1)
	df = df.loc[-flag, ]

	# 기준 컬럼을 제외하고 나머지 제거
	df = df[[_col]]
    
	# df에 STD-YM 컬럼 생성하여 인덱스에서 년도-월 데이터 추출하여 대입
	df['STD-YM'] = df.index.strftime('%Y-%m')

	return df		

In [185]:
df = pd.read_csv('../../csv/AAPL.csv')
ym_df = create_YM(df)

2. 월말의 데이터프레임 생성하는 함수 (create_last_month)
	- 매개변수 4개
		- _df: STD-YM 컬럼이 생성된 데이터프레임 (필수)
		- _start: 투자 시작시간 (기본값 '2010-01-01')
		- _end: 투자 종료시간 (기본값 현재 시간)
		- _momentum: 모멘텀 기간 (기본값 12)
			- 6으로 지정한다면 전월과 6개월 전 데이터 이용
	1. 월말의 데이터만 모아서 새로운 데이터프레임 생성
	2. 전월의 기준이 되는 컬럼의 데이터를 BF1 컬럼에 대입
	3. BF2 컬럼을 생성해 모멘텀 기간만큼 전의 데이터 대입
	4. 결측치는 0으로 대체
	5. 투자의 시작 시간과 종료 시간으로 필터링
	6. 수정된 데이터프레임 반환

In [ ]:
def create_last_month(
		_df, 
		_start = '2010-01-01', 
		_end = datetime.now(), 
		_momentum = 12):
		
	# 기준이 되는 컬럼 이름을 변수에 저장
	col = _df.columns[0]

	# 월말의 데이터만 모아 생성할 새로운 데이터프레임
	# 월말의 기준 : STD-YM의 현재와 다음행 데이터가 다른 경우
	flag = _df['STD-YM'] != _df.shift(-1)['STD-YM']
	df = _df.loc[flag, ]

	# 전월의 데이터 생성
	df['BF1'] = df.shift(1)[col].fillna(0)
	# _momentum 개월 전의 데이터 생성
	df['BF2'] = df.shift(_momentum)[col].fillna(0)
	
	df = df.loc[_start : _end, ]
	
	return df

In [187]:
bf_df = create_last_month(ym_df)

3. 거래내역을 추가하는 함수 (create_rtn)
	- 매개변수 
		- _df1 : create_YM() 함수의 결과(필수)
		- _df2 : create_last_month() 함수의 결과 (필수)
		- _score : 모멘텀 스코어 (기본값 1)
	1. _df1의 복사본 생성 -> df
	2. df에 trade 컬럼을 생성해 '' 대입
	3. _df2의 데이터를 이용해 momentum_index를 생성
	4. momentum_index 값에 따라 df 거래 내역 추가
	5. df에 rtn 컬럼을 생성해 1을 대입
	6. df의 trade를 기준으로 수익률 계산해 rtn 컬럼에 대입
	7. 누적수익률 계산하여 acc_rtn 컬럼을 생성해 대입
	8. df, 최종수익률 반환

In [188]:
def create_rtn(_df1, _df2, _score = 1):
	# 데이터프레임 복사본 생성
	df = _df1.copy()
	
	# 거래 내역 컬럼 생성
	df['trade'] = ''
	df['rtn'] = 1
	
	# 거래내역 추가
	# _df2(create_last_month())를 이용해 momentum_index를 구함
	for idx in _df2.index:
		signal= ''

		# 절대 모멘텀 인덱스 계산
		momentum_index = \
			(_df2.loc[idx, 'BF1'] / _df2.loc[idx, 'BF2']) - _score

		# 모멘텀 인덱스가 0보다 크고 무한대가 아닌 경우
		flag = (momentum_index > 0) & (momentum_index != np.inf)
		
		# 조건(flag)이 참이라면
		if flag:
			signal = 'buy'
		# print(f'날짜: {idx}, 모멘텀인덱스: {momentum_index}, signal: {signal}')
		# df에 구매내역 추가
		df.loc[idx: , 'trade'] = signal
	
	# 기준이 되는 컬럼 이름을 변수에 저장
	col = df.columns[0]

	# 수익률 계산
	for idx in df.index:
		# 매수의 조건식: 전날의 trade가 ''이고, 오늘의 trade가 'buy'라면
		if (df.shift().loc[idx, 'trade'] == '') & (df.loc[idx, 'trade'] == 'buy'):
			buy = df.loc[idx, col]		# 구매가는 해당 날짜의 수정종가(Adj Close)
			print(f'매수일: {idx}, 매수가: {buy}')
		# 매도의 조건식: 전날의 trade가 'buy'이고, 오늘의 trade가 ''라면
		elif (df.shift().loc[idx, 'trade'] == 'buy') & (df.loc[idx, 'trade'] == ''):
			sell = df.loc[idx, col]		# 매도가는 해당 날짜의 수정종가(Adj Close)
			rtn = sell / buy
			df.loc[idx, 'rtn'] = rtn
			print(f'매도일: {idx}, 매도가: {sell}, 수익률: {rtn}')
	# 누적곱(누적수익률)
	df['acc_rtn'] = df['rtn'].cumprod()
	acc_rtn = df.iloc[-1, -1]	

	return df, acc_rtn


In [189]:
create_rtn(ym_df, bf_df)

매수일: 2010-01-29 00:00:00, 매수가: 24.035734
매도일: 2013-02-28 00:00:00, 매도가: 56.055256, 수익률: 2.3321632699047177
매수일: 2013-12-31 00:00:00, 매수가: 72.612869
매도일: 2016-01-29 00:00:00, 매도가: 91.581261, 수익률: 1.2612263123772178
매수일: 2016-12-30 00:00:00, 매수가: 111.392426
매도일: 2019-01-31 00:00:00, 매도가: 165.093445, 수익률: 1.4820886026847104
매수일: 2019-03-29 00:00:00, 매수가: 189.221313
매도일: 2019-06-24 00:00:00, 매도가: 199.169998, 수익률: 1.052576979000246


(             Adj Close   STD-YM trade       rtn   acc_rtn
 Date                                                     
 1980-12-12    0.410525  1980-12        1.000000  1.000000
 1980-12-15    0.389106  1980-12        1.000000  1.000000
 1980-12-16    0.360548  1980-12        1.000000  1.000000
 1980-12-17    0.369472  1980-12        1.000000  1.000000
 1980-12-18    0.380182  1980-12        1.000000  1.000000
 ...                ...      ...   ...       ...       ...
 2019-06-18  198.449997  2019-06   buy  1.000000  4.359394
 2019-06-19  197.869995  2019-06   buy  1.000000  4.359394
 2019-06-20  199.460007  2019-06   buy  1.000000  4.359394
 2019-06-21  198.779999  2019-06   buy  1.000000  4.359394
 2019-06-24  199.169998  2019-06        1.052577  4.588598
 
 [9713 rows x 5 columns],
 np.float64(4.588597970528201))